In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import Row

spark = SparkSession \
    .builder \
    .appName("Python Spark SQL another example") \
    .getOrCreate()



Setting spark.hadoop.yarn.resourcemanager.principal to user050
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/22 13:09:26 WARN JettyUtils: GET /jobs/ failed: java.util.NoSuchElementException: Failed to get the application information. If you are starting up Spark, please wait a while until it's ready.
java.util.NoSuchElementException: Failed to get the application information. If you are starting up Spark, please wait a while until it's ready.
	at org.apache.spark.status.AppStatusStore.applicationInfo(AppStatusStore.scala:51)
	at org.apache.spark.ui.jobs.AllJobsPage.render(AllJobsPage.scala:276)
	at org.apache.spark.ui.WebUI.$anonfun$attachPage$1(WebUI.scala:90)
	at org.apache.spark.ui.JettyUtils$$anon$1.doGet(JettyUtils.scala:81)
	at javax.servlet.http.HttpServlet.service(HttpServlet.java:687)
	at javax.servlet.http.Ht

In [2]:
flights=spark.read.options(header='True',inferSchema='True',delimiter=',')\
    .csv("files/521972167_T_ONTIME.csv")
flights.createOrReplaceTempView("flights")
totalFlights = spark.sql("SELECT count(*) AS total FROM flights")
totalFlights.show()


24/01/22 13:09:49 WARN TaskSchedulerImpl: Initial job has not accepted any resources; check your cluster UI to ensure that workers are registered and have sufficient resources
24/01/22 13:09:55 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+------+
| total|
+------+
|502457|
+------+



In [3]:
flights.createOrReplaceTempView("flights")
totalFlights = spark.sql("SELECT count(*) AS total FROM flights")
totalFlights.show()

24/01/22 13:12:01 WARN ExecutorPodsSnapshotsStoreImpl: Exception when notifying snapshot subscriber.
io.fabric8.kubernetes.client.KubernetesClientException: Failure executing: POST at: https://172.20.0.1/api/v1/namespaces/mlx-user-2/pods. Message: pods "cdsw-lc1r4zramizhej5f-exec-3" already exists. Received status: Status(apiVersion=v1, code=409, details=StatusDetails(causes=[], group=null, kind=pods, name=cdsw-lc1r4zramizhej5f-exec-3, retryAfterSeconds=null, uid=null, additionalProperties={}), kind=Status, message=pods "cdsw-lc1r4zramizhej5f-exec-3" already exists, metadata=ListMeta(_continue=null, remainingItemCount=null, resourceVersion=null, selfLink=null, additionalProperties={}), reason=AlreadyExists, status=Failure, additionalProperties={}).
	at io.fabric8.kubernetes.client.dsl.base.OperationSupport.requestFailure(OperationSupport.java:681)
	at io.fabric8.kubernetes.client.dsl.base.OperationSupport.assertResponseCode(OperationSupport.java:620)
	at io.fabric8.kubernetes.client.ds

+------+
| total|
+------+
|502457|
+------+



In [4]:
# Top 10 performances airports on weekends
top10airports = spark.sql("SELECT origin, count(*) AS total_departures " +
  "FROM flights WHERE DAY_OF_WEEK > 5 GROUP BY origin " +
  "ORDER BY total_departures DESC LIMIT 10")
top10airports.show()

+------+----------------+
|origin|total_departures|
+------+----------------+
|   ATL|           10012|
|   ORD|            6579|
|   DEN|            6396|
|   LAX|            6026|
|   DFW|            5268|
|   SFO|            4601|
|   PHX|            4320|
|   SEA|            4105|
|   LAS|            4072|
|   MCO|            3682|
+------+----------------+



In [5]:
# Top 5 delayed airports
top5delayed = spark.sql("SELECT origin, count(1) as cnt " +
  "FROM flights WHERE ARR_DELAY >= 15 GROUP BY origin " +
  "ORDER BY cnt DESC LIMIT 5")
top5delayed.show()

+------+----+
|origin| cnt|
+------+----+
|   ATL|7923|
|   ORD|6425|
|   DFW|5254|
|   LAX|5136|
|   DEN|5002|
+------+----+



In [6]:
# Top 5 big delay airports
top5BigDelay = spark.sql("SELECT origin, count(1) as cnt " +
  "FROM flights WHERE ARR_DELAY >= 60 GROUP BY origin " +
  "ORDER BY cnt DESC LIMIT 5")
top5BigDelay.show()

+------+----+
|origin| cnt|
+------+----+
|   ATL|3421|
|   ORD|2811|
|   DFW|2100|
|   DEN|1793|
|   LAX|1628|
+------+----+



In [7]:
# Top 5 airports with more cancellations
top5Cancellations = spark.sql("SELECT origin, count(1) as cnt " +
  "FROM flights WHERE CANCELLED = 1 GROUP BY origin " +
  "ORDER BY cnt DESC LIMIT 5")
top5Cancellations.show()

+------+---+
|origin|cnt|
+------+---+
|   ORD|763|
|   EWR|564|
|   LGA|444|
|   DFW|280|
|   MDW|275|
+------+---+



In [8]:
# Top 5 popular routes
top5routes = spark.sql("SELECT origin, dest, count(1) as cnt " +
  "FROM flights GROUP BY  origin, dest " +
  "ORDER BY cnt DESC LIMIT 5")
top5routes.show()

+------+----+----+
|origin|dest| cnt|
+------+----+----+
|   SFO| LAX|1526|
|   LAX| SFO|1506|
|   JFK| LAX|1127|
|   LAX| JFK|1127|
|   SEA| LAX|1047|
+------+----+----+

